In [ ]:
import pandas as pd
import os.path 
import csv
from interface import input_path, input_load_date
from datetime import datetime, timedelta    
from routes_export import export_new_routes_files
pd.options.mode.chained_assignment = None
from interface import * 
from fix_routes_converter  import *
from flex_routes_converter import *
from tl_routes_converter import *
from HnN_routes_converter import *
from routes_export import *
from time import sleep

In [ ]:
PATH = r"C:\Users\ROKOTYEV\Desktop\Формат - Гибкие маршруты от Агенств.xlsx"

df_rtm = pd.read_excel(PATH, sheet_name='MergeDATA', usecols='A:R',  dtype='str')#, engine = 'pyxlsb')
df_rtm = df_rtm.replace(r'\n','', regex=True)

In [ ]:
renames = {'Тип маршрута'                  :  'ROUTE_TYPE', 
           'ТТ SAP Код'                    :  'SHIP_TO', 
           'Наименование маршрута'         :  'ROUTE_NAME', 
           'Ответственный'                 :  'ROUTE_ID', 
           	'Длительность визита (минут)'  :  'VISIT_DURATION',
            'Агенство'                     :  'AGENCY_NAME',
           'Weekly Frequency'              :  'FREQUENCY_RTM',
           'РОЛЬ'                          :  'ROLE',
           'Форма заказа'                  :  'ORDER_TYPE',
           'СЦЕНАРИЙ'                      :  'SCENARIO'}    
df_rtm.rename(columns=renames, inplace=True)
df_rtm['SHIP_TO'] = df_rtm['SHIP_TO'].astype('float').astype('int').astype('str')
df_rtm['VISIT_DURATION'] = df_rtm['VISIT_DURATION'].astype('float').astype('int')

In [ ]:
columns_list = ['ROUTE_TYPE','SHIP_TO','ROUTE_NAME','ROUTE_ID','FREQUENCY_RTM','AGENCY_NAME','VISIT_DURATION','ROLE','ORDER_TYPE','SCENARIO',
                'ПН1', 'ВТ1', 'СР1', 'ЧТ1', 'ПТ1', 'СБ1', 'ВС1' ]
df_rtm = df_rtm[columns_list]
filt = df_rtm['ROUTE_TYPE'] == 'Мерчандайзер'
df_rtm = df_rtm[filt]
filt = (df_rtm['ORDER_TYPE'] == 'FLEX')  &  (df_rtm['AGENCY_NAME'] == str('OPEN'))   
df_rtm = df_rtm[filt]

In [ ]:
df_rtm_flex = pd.melt(df_rtm, id_vars=['ROUTE_TYPE','SHIP_TO','ROUTE_NAME','ROUTE_ID','FREQUENCY_RTM','AGENCY_NAME','VISIT_DURATION','ROLE','ORDER_TYPE','SCENARIO'])
renames = {'variable'  :  'DAY_NAME', 
           'value'     :  'VISIT_NUMBER'} 
df_rtm_flex.rename(columns=renames, inplace=True)
filt = ((df_rtm_flex['VISIT_NUMBER'] == '1') + (df_rtm_flex['VISIT_NUMBER'] == 1))
df_rtm_flex = df_rtm_flex[filt]
df_rtm_flex['WEEK_1234_ORDER'] = df_rtm_flex['DAY_NAME'].str[-1]
df_rtm_flex['WEEK_1234_ORDER'] = df_rtm_flex['WEEK_1234_ORDER'].astype('int')
df_rtm_flex['DAY_NAME']    = df_rtm_flex['DAY_NAME'].str[0:2]
df_rtm_flex['DAY_NAME']    = df_rtm_flex['DAY_NAME'].astype('str')
df_rtm_flex['EXT_ROUTE_ID'] = df_rtm_flex['ROUTE_NAME'].str.split('_').str[-1].str.strip()
df_rtm_flex['ROW_NUMBER'] = df_rtm_flex.groupby(by=['ROUTE_NAME','DAY_NAME','WEEK_1234_ORDER' ]).cumcount()+1
df_rtm_flex.loc[df_rtm_flex['VISIT_NUMBER'] == 1, ['VISIT_NUMBER']] = df_rtm_flex['ROW_NUMBER']
df_rtm_flex['SHIP_TO'] = df_rtm_flex['SHIP_TO'].astype('float').astype('int').astype('str')

In [ ]:
--------------------------

In [57]:
PATH = r"C:\Users\ROKOTYEV\Desktop\Формат - Маршрут тимлида.xlsx"
df_rtm = pd.read_excel(PATH, sheet_name='MergeDATA', usecols='A:R',  dtype='str')#, engine = 'pyxlsb')
df_rtm = df_rtm.replace(r'\n','', regex=True)

In [58]:
renames = {'Тип маршрута'                  :  'ROUTE_TYPE', 
           'ТТ SAP Код'                    :  'SHIP_TO', 
           'Наименование маршрута'         :  'ROUTE_NAME', 
           'Ответственный'                 :  'ROUTE_ID', 
           	'Длительность визита (минут)'  :  'VISIT_DURATION',
            'Агенство'                     :  'AGENCY_NAME',
           'Weekly Frequency'              :  'FREQUENCY_RTM',
           'РОЛЬ'                          :  'ROLE',
           'Форма заказа'                  :  'ORDER_TYPE',
           'СЦЕНАРИЙ'                      :  'SCENARIO'}    
df_rtm.rename(columns=renames, inplace=True)
df_rtm['SHIP_TO'] = df_rtm['SHIP_TO'].astype('float').astype('int').astype('str')
df_rtm['VISIT_DURATION'] = df_rtm['VISIT_DURATION'].astype('float').astype('int') 

In [59]:
columns_list = ['ROUTE_TYPE','SHIP_TO','ROUTE_NAME','ROUTE_ID','FREQUENCY_RTM','AGENCY_NAME','VISIT_DURATION','ROLE','ORDER_TYPE','SCENARIO',
                'ПН1', 'ВТ1', 'СР1', 'ЧТ1', 'ПТ1', 'СБ1', 'ВС1' ]
df_rtm = df_rtm[columns_list]
df_rtm['ROUTE_TYPE'] = df_rtm['ROUTE_TYPE'].str.replace(' ', '').str.lower().str.strip()
df_rtm = df_rtm.query("ROUTE_TYPE in ('тимлид', 'супервайзер')", engine='python')
filt = df_rtm['AGENCY_NAME'] == str('OPEN')  
df_rtm = df_rtm[filt]

In [60]:
df_rtm = df_rtm.groupby(by=['AGENCY_NAME', 'ROUTE_NAME', 'SHIP_TO']).agg('size').reset_index().drop(columns=0)
#df_rtm['VISIT_NUMBER'] = df_rtm['VISIT_NUMBER'].astype('str')
#df_rtm.loc[df_rtm['VISIT_NUMBER'] != '0', ['VISIT_NUMBER']] = '1'
df_rtm['EXT_ROUTE_ID'] = df_rtm['ROUTE_NAME'].str.split('_').str[-1].str.strip()
df_rtm['EXT_ROUTE_ID'] = '0000' + df_rtm['EXT_ROUTE_ID'].astype('str')
df_rtm['EXT_ROUTE_ID'] = df_rtm['EXT_ROUTE_ID'].str[-4:]
df_rtm['ROUTE_TYPE'] = 'Тим Лид' 
df_rtm['FIRST_VISIT_DATE'] = '2024-12-30'
df_rtm['REPEAT_DAYS']    = '0'
df_rtm['VISIT_NUMBER'] = '1' 
df_rtm['VISIT_DURATION'] = '100' 
df_rtm['PHOTOS_TARGET'] = '5' 
df_rtm['DOCUMENTS_TARGET'] = '5' 
df_rtm['PHOTOAUDIT_TARGET'] = '1' 
df_rtm['END_DATE'] = '2024-12-31'
df_routes = df_rtm[['AGENCY_NAME','ROUTE_NAME','EXT_ROUTE_ID','ROUTE_TYPE','SHIP_TO','FIRST_VISIT_DATE','REPEAT_DAYS','VISIT_NUMBER','VISIT_DURATION', 'PHOTOS_TARGET','DOCUMENTS_TARGET','PHOTOAUDIT_TARGET', 'END_DATE']]

In [61]:
df_rtm

,AGENCY_NAME,ROUTE_NAME,SHIP_TO,EXT_ROUTE_ID,ROUTE_TYPE,FIRST_VISIT_DATE,REPEAT_DAYS,VISIT_NUMBER,VISIT_DURATION,PHOTOS_TARGET,DOCUMENTS_TARGET,PHOTOAUDIT_TARGET,END_DATE
0,OPEN,TL_TestTerritory_3,850024459,0003,Тим Лид,2024-12-30,0,1,100,5,5,1,2024-12-31
1,OPEN,TL_TestTerritory_3,850030525,0003,Тим Лид,2024-12-30,0,1,100,5,5,1,2024-12-31
2,OPEN,TL_TestTerritory_3,850055190,0003,Тим Лид,2024-12-30,0,1,100,5,5,1,2024-12-31
3,OPEN,TL_TestTerritory_3,850100250,0003,Тим Лид,2024-12-30,0,1,100,5,5,1,2024-12-31
4,OPEN,TL_TestTerritory_3,850169360,0003,Тим Лид,2024-12-30,0,1,100,5,5,1,2024-12-31
5,OPEN,TL_TestTerritory_3,850170703,0003,Тим Лид,2024-12-30,0,1,100,5,5,1,2024-12-31
6,OPEN,TL_TestTerritory_3,850179881,0003,Тим Лид,2024-12-30,0,1,100,5,5,1,2024-12-31
7,OPEN,TL_TestTerritory_3,850186507,0003,Тим Лид,2024-12-30,0,1,100,5,5,1,2024-12-31
8,OPEN,TL_TestTerritory_3,850243076,0003,Тим Лид,2024-12-30,0,1,100,5,5,1,2024-12-31
9,OPEN,TL_TestTerritory_3,850288526,0003,Тим Лид,2024-12-30,0,1,100,5,5,1,2024-12-31
